# Convolutional Neural Network

### Importing the libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import EarlyStopping
import time
import os

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [6]:
# Apply data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [8]:
# Preprocessing the test set
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [11]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [13]:
cnn.add(Input(shape=(64, 64, 3)))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 2 - Pooling

In [15]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [17]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [19]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [21]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [23]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [26]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [28]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()

cnn.fit(x=training_set, validation_data=test_set, epochs = 30, callbacks=[early_stopping])

end_time = time.time()

total_time = end_time - start_time

print(f"Total training time: {total_time:.2f} seconds")
print(f"Total training time: {total_time / 60:.2f} minutes")



C:\Users\soumitro_uni\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.5602 - loss: 0.6893

C:\Users\soumitro_uni\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 223ms/step - accuracy: 0.5604 - loss: 0.6891 - val_accuracy: 0.6515 - val_loss: 0.6296
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - accuracy: 0.6821 - loss: 0.5944 - val_accuracy: 0.6900 - val_loss: 0.5967
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 58s 230ms/step - accuracy: 0.7113 - loss: 0.5586 - val_accuracy: 0.7295 - val_loss: 0.5359
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 58s 231ms/step - accuracy: 0.7474 - loss: 0.5181 - val_accuracy: 0.7615 - val_loss: 0.5082
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.7544 - loss: 0.4953 - val_accuracy: 0.7390 - val_loss: 0.5356
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 58s 232ms/step - accuracy: 0.7801 - loss: 0.4683 - val_accuracy: 0.7485 - val_loss: 0.5043
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.7871 - loss: 0.4539 - val_accuracy: 0.7705 - val_loss: 0.4795
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 55s 221ms/step - accuracy: 0.8015 - loss: 0.4276 - val

## Part 4 - Making a single prediction

In [30]:
single_prediction_dir = 'dataset/single_prediction'

for img_file in os.listdir(single_prediction_dir):
    img_path = os.path.join(single_prediction_dir, img_file)
    test_image = image.load_img(img_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0) / 255.0
    
    prediction = cnn.predict(test_image)
    if prediction[0][0] > 0.5:
        print(f"{img_file} is a Dog")
    else:
        print(f"{img_file} is a Cat")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
cat_or_dog_1.jpg is a Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
cat_or_dog_2.jpg is a Cat


### Github Link: https://github.com/soumitro114/CNN-for-Image-Classification

### Medium Link: https://medium.com/@soumitro1604114/building-a-convolutional-neural-network-for-cat-vs-dog-classification-40d44f45d4c3